In [72]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image

# global variables
WIDTH = 1280
HEIGHT = 853
WINDOW = "Volume Measurment Tests"
THRESH_MAX = 150
MAX_VAL = 150

# creating parameter window
# def empty(a):
#     pass
# cv.namedWindow("Parameters", cv.WINDOW_NORMAL)
# cv.resizeWindow("Parameters", 600, 100)
# cv.createTrackbar("Threshold", "Parameters", 0, 255, empty)
# cv.createTrackbar("Max Value", "Parameters", 0, 255, empty)

# importing raw image
raw = Image.open("IMG_0367.CR2").convert('RGB')

#converting it to a numpy.array for opencv
img = np.array(raw) # in RGB
img = cv.cvtColor(img, cv.COLOR_RGB2BGR)

# converting img to grayscale
gray = cv.cvtColor(img, cv.COLOR_RGB2GRAY)

# adding trackbars
# def thresh_trackbar(val):
#     # applying binary threshold
#     return cv.threshold(gray, val, 250, cv.THRESH_BINARY)
# def maxval_trackbar(val):
#     # applying binary threshold
#     return cv.threshold(gray, val, 250, cv.THRESH_BINARY)

# threshold = cv.getTrackbarPos("Threshold", "Parameters")
# maxval = cv.getTrackbarPos("Max Value", "Parameters")


# trackbar_name = 'Alpha x %d' % THRESH_MAX
# cv.createTrackbar(trackbar_name, WINDOW , 0, THRESH_MAX, on_trackbar)


# applying binary threshold
ret, thresh = cv.threshold(gray, 15, 100, cv.THRESH_BINARY)
thresh_blur = cv.GaussianBlur(thresh, (11, 11), 0)
contours, _ = cv.findContours(thresh_blur, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)

blurred = cv.GaussianBlur(gray, (101,101), 0)

edges = cv.Canny(blurred, 1, 6)
        
        
def max_contour(list):
    largest = list[0]
    for cnt in list:
        if cnt.shape[0] > largest.shape[0]:
            largest = cnt
    return largest



In [68]:
# Find the rotated rectangles and ellipses for each contour
minRect = [None]*len(contours)
minEllipse = [None]*len(contours)
for i, c in enumerate(contours):
    minRect[i] = cv.minAreaRect(c)
    if c.shape[0] > 30:
        minEllipse[i] = cv.fitEllipse(c)

        
# Find fitting line
lines = [None]*len(contours)
for i, cont in enumerate(contours):
    if cont.shape[0] > 30:
        lines[i] = cv.fitLine(cont, cv.DIST_L2,0,0.01,0.01) # [vx,vy,x,y]

In [69]:
ballon_contour = max_contour(contours)
ballon_ellipse = cv.fitEllipse(ballon_contour)

# draw the contour and ellipse
img_overlay = img.copy()
cv.drawContours(img_overlay, ballon_contour, -1, (0,255,0), 5, lineType=cv.LINE_AA)
cv.ellipse(img_overlay, ballon_ellipse, (0,0,255), 5)
# for i, cont in enumerate(contours):
#     cv.drawContours(img_overlay, contours, -1, (0, 255, 0), 5, lineType=cv.LINE_AA)
#     if cont.shape[0] > 30:
#         #ellipses
#         cv.ellipse(img_overlay, minEllipse[i], (0, 0, 255), 5) 


cv.namedWindow(WINDOW, cv.WINDOW_NORMAL)
cv.imshow(WINDOW, img_overlay)
cv.resizeWindow(WINDOW, WIDTH, HEIGHT)
cv.waitKey(0)
cv.imwrite('Balloon with Contours and Fitted Ellipse.png', img_overlay)
cv.destroyAllWindows()

In [46]:
# draw the contours and ellipses
img_overlay = img.copy()
for i, cont in enumerate(contours):
    cv.drawContours(img_overlay, contours, -1, (0, 255, 0), 5, lineType=cv.LINE_AA)
    if cont.shape[0] > 30:
        #ellipses
        cv.ellipse(img_overlay, minEllipse[i], (0, 0, 255), 5) 


cv.namedWindow(WINDOW, cv.WINDOW_NORMAL)
cv.imshow(WINDOW, img_overlay)
cv.resizeWindow(WINDOW, WIDTH, HEIGHT)
cv.waitKey(0)
cv.imwrite('Balloon with Contours and Fitted Ellipse.png', img_overlay)
cv.destroyAllWindows()

In [49]:
# draw the contours and lines
img_overlay = img.copy()
for i, cont in enumerate(contours):
    if cont.shape[0] > 5000:
        cv.drawContours(img_overlay, cont, -1, (0, 255, 0), 5, lineType=cv.LINE_AA)
        vx, vy, x, y = lines[i]
        left = int((-x * vy/vx) + y)
        right = int(((img_overlay.shape[1]-x) *vy/vx) + y)
        cv.line(img_overlay, (img_overlay.shape[1]-1, right), (0, left), (255, 0, 0), 5)

cv.namedWindow(WINDOW, cv.WINDOW_NORMAL)
cv.imshow(WINDOW, img_overlay)
cv.resizeWindow(WINDOW, WIDTH, HEIGHT)
cv.waitKey(0)
cv.destroyAllWindows()
cv.imwrite('Balloon with Contours and Fitted Line.png', img_overlay)

True